In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import all necessary modules first
import aicons.bayesbrainGPT.bayes_brain_ref
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.linear_utility
import aicons.bayesbrainGPT.utility_function
import aicons.definitions.aicon

# Now we can reload them
importlib.reload(aicons.bayesbrainGPT.bayes_brain_ref)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function)  # Reload the package first
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT)
importlib.reload(aicons)

# Import the classes we need
from aicons.definitions.aicon import AIcon

In [2]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


### Defining Action Space

In [3]:
# Define an action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [4]:
print(aicon.brain.action_space.raw_print()) 

dimensions: ['google_budget: [0.0, 1000.0] step=100.0', 'facebook_budget: [0.0, 1000.0] step=100.0']
constraints: ['<function create_marketing_ads_space.<locals>.budget_sum_constraint at 0x15b0b7ba0>']
actions: [{'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 100.0, 'facebook_budget': 900.0}, {'google_budget': 400.0, 'facebook_budget': 600.0}, {'google_budget': 200.0, 'facebook_budget': 800.0}, {'google_budget': 200.0, 'facebook_budget': 800.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google_budget': 700.0, 'facebook_budget': 300.0}, {'google_budget': 500.0, 'facebook_budget': 500.0}, {'google

### Defining Utility Function

In [5]:
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)


Utility Function: LinearUtility: Σ(weights * values) where weights = [tf.Tensor(0.6, shape=(), dtype=float32), tf.Tensor(0.4, shape=(), dtype=float32)]


### Defining State Rep State

In [6]:
# Add a conversion rate factor (must be between 0 and 1)
aicon.add_state_factor(
    name='conversion_rate',
    factor_type='continuous',
    value=0.05,
    uncertainty=0.01,
    constraints={'lower': 0.0, 'upper': 1.0}
)

# Add a budget factor (must be non-negative)
aicon.add_state_factor(
    name='daily_budget',
    factor_type='continuous',
    value=1000.0,
    uncertainty=100.0,
    constraints={'lower': 0.0}  # Only lower bound specified
)

# Add a categorical factor (no constraints needed)
aicon.add_state_factor(
    name='primary_channel',
    factor_type='categorical',
    value='google',
    categories=['google', 'facebook', 'twitter'],
    probs=[0.7, 0.2, 0.1]
)

AttributeError: 'BayesianState' object has no attribute 'add_latent_variable'

### Perception

In [7]:
# Try to update beliefs without any sensors
print("Current state before update:")
print(aicon.get_state())

Current state before update:
{'base_conversion_rate': 0.05, 'primary_channel': 'google'}
